In [3]:
!pip install pandas cuid

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'cuid' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'cuid'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4806 sha256=29262633b8475b28d4f747d1dddb8acb2d87ef25c02e1c4d58d96443aad5f72e
  Stored in directory: /home/hy/.cache/pip/wheels/09/7f/e3/28d5914b5c9537cc9223a8a5ca0686b57e83e9825c16263560
Successfully built cuid


In [4]:
import pandas as pd
import re
from cuid import cuid

In [6]:
input_path = "./data/all-staff-survey-data.secret.csv"
df = pd.read_csv(input_path)

In [7]:
df["comment_id"] = [cuid() for _ in range(len(df))]

In [8]:
def split_multiline_rows(row):
    """Split a comment into multiple rows if it contains newline characters."""
    comments = str(row["comment"]).split("\n")
    new_rows = []
    for c in comments:
        c = c.strip()
        if c:  # skip empty lines
            new_row = row.copy()
            new_row["comment"] = c
            new_rows.append(new_row)
    return new_rows

expanded_rows = []
for _, row in df.iterrows():
    expanded_rows.extend(split_multiline_rows(row))

df_expanded = pd.DataFrame(expanded_rows)

In [9]:
def clean_comment(text):
    if not isinstance(text, str):
        return text
    # Remove patterns like "1.", "1. ", "- ", etc. from start
    cleaned = re.sub(r'^\s*[-–•]?\s*\d*\.?\s*', '', text)
    return cleaned.strip()

df_expanded["comment"] = df_expanded["comment"].apply(clean_comment)

In [ ]:
df_expanded.reset_index(drop=True, inplace=True)
display(df_expanded.head())

,comment,criteria,faculty,academic_degree,comment_id
0,Phòng KHTC là phòng làm việc gây mất thiện cảm...,Trường có nhiều kênh để tiếp nhận ý kiến đóng ...,KTMT,KS,cmge96rlu0000hrkw4vxplxe7
1,không có đủ chuyên môn về tài chính để trả lời,"Thầy/ Cô hài lòng về năng lực quản lý, chuyên ...",HTTT,TS,cmge96rlu0001hrkwfpqqxq0u
2,Không có đầy đủ thông tin để cho ý kiến,"Thầy/ Cô hài lòng về năng lực quản lý, chuyên ...",KTMT,ThS,cmge96rlu0002hrkwfwy6tmfs
3,Không có ý kiến,Lý do Câu 2 - Câu hỏi chung,TTNN,ThS,cmge96rlu0003hrkwg5vetxa9
4,Không có ý kiến,Thầy/ Cô dự định gắn bó lâu dài trong công việ...,TTNN,ThS,cmge96rlu0004hrkwey1h9l7j


In [11]:
output_path = "./data/cleaned_data.secret.csv"
df_expanded.to_csv(output_path, index=False)

In [12]:
df_expanded

,comment,criteria,faculty,academic_degree,comment_id
0,Phòng KHTC là phòng làm việc gây mất thiện cảm...,Trường có nhiều kênh để tiếp nhận ý kiến đóng ...,KTMT,KS,cmge96rlu0000hrkw4vxplxe7
1,không có đủ chuyên môn về tài chính để trả lời,"Thầy/ Cô hài lòng về năng lực quản lý, chuyên ...",HTTT,TS,cmge96rlu0001hrkwfpqqxq0u
2,Không có đầy đủ thông tin để cho ý kiến,"Thầy/ Cô hài lòng về năng lực quản lý, chuyên ...",KTMT,ThS,cmge96rlu0002hrkwfwy6tmfs
3,Không có ý kiến,Lý do Câu 2 - Câu hỏi chung,TTNN,ThS,cmge96rlu0003hrkwg5vetxa9
4,Không có ý kiến,Thầy/ Cô dự định gắn bó lâu dài trong công việ...,TTNN,ThS,cmge96rlu0004hrkwey1h9l7j
...,...,...,...,...,...
497,Mỗi phòng ban và nhân viên của trường cần làm ...,Những điều Thầy/Cô mong muốn Nhà trường cải th...,NaN,NaN,cmge96rlv00aahrkwj7eoikmc
498,Nâng cấp tốc độ internet tốt hơn,Những điều Thầy/Cô mong muốn Nhà trường cải th...,NaN,NaN,cmge96rlv00abhrkwv60hsx8q
499,Cám ơn!,Những điều Thầy/Cô mong muốn Nhà trường cải th...,NaN,NaN,cmge96rlv00achrkwc9972dts
500,Xây dựng KPIs lượng hóa khối lượng công việc c...,Những điều Thầy/Cô mong muốn Nhà trường cải th...,NaN,NaN,cmge96rlv00adhrkw1bt5fb4k
